# Explore Buffer Extract

This notebook is for exploring and comparing various methods of doing zonal stats on a buffer around a point

fraster_extract is the fast, randomized point method. It is: 
1. Faster than typical rasterization-based methods (e.g. rasterstats)
2. More flexible and precise than scanline methods 
3. Able to use many zonalstats methods: mean, weighted mean, unique values, max, min, etc. Can expand to many more by adding functions to fraster_extract.py module

However, there are limitations. Mainly:
1. Only works as a buffer from centroid currently (not from a full, irregular shaped polygon)
2. Can only handle equal area projections in meters or WGS84 (lat/lon)

This notebook is to familiarize yourself with the different methods and the differences between them, both in terms of resulting statistics and in efficiency

In [ ]:
# Install Matplotlib and rasterstats, if not already installed
# %pip install rasterstats
# %pip install matplotlib

In [ ]:
import os
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import rasterio as rio
from rasterio.plot import plotting_extent
import geopandas as gpd
import rasterstats as rs
import pandas as pd
from shapely.geometry import Point
import timeit
import time
import matplotlib.path
import math
import fraster_extract as fe


In [ ]:
# Load the data
ds = rio.open('../data/SJER_lidarCHM.tif')
SJER_chm_data = ds.read(1, masked=True)
sjer_chm_meta = ds.profile

In [ ]:
# Create 10000 random points to buffer and get zonal stats from:
xrange = [sjer_chm_meta['transform'][2], 
          sjer_chm_meta['transform'][2]+(sjer_chm_meta['width']*sjer_chm_meta['transform'][0])]
yrange = [sjer_chm_meta['transform'][5]+(sjer_chm_meta['height']*sjer_chm_meta['transform'][4]), 
          sjer_chm_meta['transform'][5]]
          
x_coords = np.random.uniform(xrange[0]+50, xrange[1]-50, 10000)
y_coords = np.random.uniform(yrange[0]+50, yrange[1]-50, 10000)

df = pd.DataFrame({'coords':list(zip(x_coords, y_coords))})
df['coords'] = list(zip(x_coords, y_coords))
df['coords'] = df['coords'].apply(Point)

gdf = gpd.GeoDataFrame(df, geometry='coords')

# Traditional Method: Rasterstats.zonalstats

In [ ]:
# geopandas + rasterstats method
# Create a buffered polygon layer from your plot location points
def zstats_buffer(gdf, buffer_r=20):
    gdf_poly = gdf.copy()

    # Buffer each point using a 20 meter circle radius and replace the point geometry with the new buffered geometry
    gdf_poly['point_num'] = range(gdf_poly.shape[0])
    gdf_poly.geometry = gdf.geometry.buffer(buffer_r)
    # If the dir does not exist, create it
    if not os.path.isdir('../data/outputs/'):
        os.mkdir('../data/outputs/')

    # Export the buffered point layer as a shapefile to use in zonal stats
    plot_buffer_path = '../data/outputs/plot_buffer.shp'
    gdf_poly.to_file(plot_buffer_path)

    # Extract zonal stats
    zstats = rs.zonal_stats(plot_buffer_path,
                                       SJER_chm_data,
                                       affine=sjer_chm_meta['transform'],
                                       geojson_out=True,
                                       copy_properties=True,
                                       stats=["mean", "max"])
    return zstats

In [ ]:
# Standard method. See "Compare" Section for comparison. 
# Warning: this takes 30-60 sec to run
zstats = zstats_buffer(gdf, 10)
# zstats_standard_mean = gpd.GeoDataFrame.from_features(zstats)['mean']
# zstats_standard_max = gpd.GeoDataFrame.from_features(zstats)['max']

# Improved Method: Random Points

In [ ]:
# Calc mean and max
# About 4 seconds
buff_means, buff_maxes = fe.random_buffer_extract(gdf['coords'], ds, radius=10,n_sample=4000, stat='mean_max', latlon=False)

# Scanline

In [ ]:
# Indices problem, finding wrong indices

def round_mid(x, a):
    return np.floor(x / a) * a + a/2

def round_up_mid(x, a):
    return np.ceil((x-(a/2))/a) * a + a/2

def round_down_mid(x, a):
    return np.floor((x+(a/2))/a) * a - a/2

def scanline_points(target_pts, radius, pixel_dims):
    rounded_y = round_mid(target_pts[1], pixel_dims[1])
    scanline_offsets = np.linspace(-radius, radius, 
                               int(2*radius/pixel_dims[1] + 1 ))
    scanline_y = np.tile(rounded_y, (scanline_offsets.shape[0], 1)) + scanline_offsets[:,None]
    low_x = -np.sqrt(radius**2 - np.square(scanline_y - target_pts[1])) + target_pts[0]
    high_x = np.sqrt(radius**2 - np.square(scanline_y - target_pts[1])) + target_pts[0]
    low_high_sol = np.array([low_x, high_x, scanline_y])
    return low_high_sol

def interior_cells(target_pts, radius, transform):
    pixel_dims = [abs(transform.a), abs(transform.e)]
    sl_pts = scanline_points(target_pts, radius, pixel_dims)
    # Get cells based on transform
    sl_start = round_up_mid(sl_pts[0], pixel_dims[0])
    sl_end = round_down_mid(sl_pts[1], pixel_dims[0])
    invtrans = ~transform
    start_cols, row = np.floor(invtrans*(sl_start, sl_pts[2]))
    end_cols, row = np.floor(invtrans*(sl_end, sl_pts[2]))
    return np.array([start_cols, end_cols, row])

def slc_buffer(ds, target_pts, radius):
    target_cells = interior_cells(target_pts, radius, ds.transform)
    valid_indices = ~np.isnan(target_cells[0])
    cells = target_cells.astype(int)
    band = ds.read(1)
    means = np.empty(cells.shape[2])
    maxes = np.empty(cells.shape[2])
    for i in range(cells.shape[2]):
        vals = band[
            np.concatenate([np.repeat(cells[2,j, i], max(0, cells[1,j,i] - cells[0,j,i])) for j in range(cells.shape[1]) if valid_indices[j,i]]),
            np.concatenate([np.arange(cells[0,j, i],cells[1,j, i]) for j in range(cells.shape[1]) if valid_indices[j,i]])]
        means[i] = np.mean(vals)
        maxes[i] = np.max(vals)
    return means, maxes
    

In [ ]:
# Even faster, 2-3 seconds
zstats_slc_mean, zstats_slc_max =  slc_buffer(ds, np.array([gdf['coords'].x.values, gdf['coords'].y.values]), 10)

# Compare results

In [ ]:
# Standard method
zstats = zstats_buffer(gdf, 10)
zstats_standard_mean = gpd.GeoDataFrame.from_features(zstats)['mean']
zstats_standard_max = gpd.GeoDataFrame.from_features(zstats)['max']

In [ ]:
# Fully random
zstats_rand_mean, zstats_rand_max = fe.random_buffer_extract(gdf['coords'], ds, radius=10, n_sample=1000, stat='mean_max', latlon=False)

In [ ]:
# Scanline
zstats_slc_mean, zstats_slc_max =  slc_buffer(ds, np.array([gdf['coords'].x.values, gdf['coords'].y.values]), 10)

In [ ]:
### Compare default vs random
fig, ax = plt.subplots(figsize=(12, 10))

plt.scatter(gpd.GeoDataFrame.from_features(zstats)['mean'], zstats_rand_mean)
plt.title('Mean Comparison: Default vs. Random (Alg #1)', {'fontsize':24})
plt.ylabel('Random', {'fontsize':18})
plt.xlabel('Default Zonal Stats', {'fontsize':18})
plt.plot([0,100], [0,100], color='r')
plt.xlim(0, np.max(zstats_rand_mean)+2)
plt.ylim(0, np.max(zstats_rand_mean)+2)
plt.show()

fig, ax = plt.subplots(figsize=(12, 10))
plt.scatter(gpd.GeoDataFrame.from_features(zstats)['max'], zstats_rand_max)
plt.title('Max Comparison: Default vs. Random (Alg #1)', {'fontsize':24})
plt.ylabel('Random', {'fontsize':18})
plt.xlabel('Default Zonal Stats', {'fontsize':18})
plt.plot([0,100], [0,100], color='r')
plt.xlim(0, np.max(zstats_rand_max)+2)
plt.ylim(0, np.max(zstats_rand_max)+2)

plt.show()

In [ ]:
### Compare default vs slc
fig, ax = plt.subplots(figsize=(12, 10))

plt.scatter(gpd.GeoDataFrame.from_features(zstats)['mean'], zstats_slc_mean)
plt.title('Mean Comparison: Default vs. Scanline Center Points (Alg #2)', {'fontsize':24})
plt.ylabel('Scanline', {'fontsize':18})
plt.xlabel('Default Zonal Stats', {'fontsize':18})
plt.plot([0,100], [0,100], color='r')
plt.xlim(0, np.max(zstats_rand_mean)+2)
plt.ylim(0, np.max(zstats_rand_mean)+2)
plt.show()

fig, ax = plt.subplots(figsize=(12, 10))
plt.scatter(gpd.GeoDataFrame.from_features(zstats)['max'], zstats_slc_max)
plt.title('Max Comparison: Default vs. Scanline Center Points (Alg #2)', {'fontsize':24})
plt.ylabel('Scanline', {'fontsize':18})
plt.xlabel('Default Zonal Stats', {'fontsize':18})
plt.plot([0,100], [0,100], color='r')
plt.xlim(0, np.max(zstats_rand_max)+2)
plt.ylim(0, np.max(zstats_rand_max)+2)

plt.show()

# Time plots

In [ ]:
# Time it all
t_zonal = timeit.Timer(lambda: zstats_buffer(gdf, 10))
t_zonal_time = t_zonal.timeit(number=5)
print('default', t_zonal_time)
t_random  = timeit.Timer(lambda: fe.random_buffer_extract(gdf['coords'], ds, radius=10, n_sample=10000, stat='mean_max', latlon=False))
t_random_time = t_random.timeit(number=5)
print('random', t_random_time)
t_scan = timeit.Timer(lambda: slc_buffer(ds, np.array([gdf['coords'].x.values, gdf['coords'].y.values]), 10))
t_scan_time = t_scan.timeit(number=5)
print('scanline', t_scan_time)

In [ ]:
times = np.array([t_zonal_time, t_random_time, t_scan_time])/5
fig, ax = plt.subplots(figsize=(15, 15))
ax.set_axisbelow(True)
ax.yaxis.grid()
plt.bar(range(3), times)
plt.xticks(range(3), ['Default', 'Random', 'Scanline'], fontsize=24)
plt.yticks([0, 5, 10, 15, 20], fontsize=20)
plt.title('Runtime Comparison (sec)', fontsize=28)
plt.ylabel('Runtime (s)', fontsize=24)


# What's going on here? 
## Explore how the different approaches work through plots

In [ ]:
target_pt = [256500,4111000.5]

In [ ]:
# Base
gdf_single = gdf.copy().loc[0:2]
gdf_single.loc[0,'coords'] = Point(target_pt[0], target_pt[1])
gdf_single_buff = gdf_single.copy()
gdf_single_buff.geometry = gdf_single.coords.buffer(10)
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(SJER_chm_data,
          # Here you must set the spatial extent or else the data will not line up with your geopandas layer
          extent=plotting_extent(ds),
          cmap='Greys')

gdf_single_buff.plot(ax=ax, facecolor='none', edgecolor='r', linewidth=2);
gdf_single.plot(ax=ax, markersize=40, color='r')
plt.xlim(target_pt[0]-12, target_pt[0]+12)
plt.ylim(target_pt[1]-12, target_pt[1]+12)

plt.show()

In [ ]:
# Heavy overlap: each pixel is bigger part of radius
gdf_single = gdf.copy().loc[0:2]
gdf_single.loc[0,'coords'] = Point(target_pt[0]+4, target_pt[1]-7)
gdf_single_buff_small = gdf_single.copy()
gdf_single_buff_small.geometry = gdf_single.coords.buffer(1.3)
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(SJER_chm_data,
          # Here you must set the spatial extent or else the data will not line up with your geopandas layer
          extent=plotting_extent(ds),
          cmap='Greys')

gdf_single_buff_small.plot(ax=ax, facecolor='none', edgecolor='r', linewidth=2);
gdf_single.plot(ax=ax, markersize=40, color='r')
plt.xlim(target_pt[0]+4-1.5, target_pt[0]+4+1.5)
plt.ylim(target_pt[1]-7-1.5, target_pt[1]-7+1.5)

plt.show()

In [ ]:
# Totally Random sample
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(SJER_chm_data,
          # Here you must set the spatial extent or else the data will not line up with your geopandas layer
          extent=plotting_extent(ds),
          cmap='Greys')

pts = fe._stats._generate_points(n=4000, radius=10)
plt.plot(pts[0] + target_pt[0], pts[1] + target_pt[1], 'bo', markersize=2)
gdf_single_buff.plot(ax=ax, facecolor='none', edgecolor='r', linewidth=2);
gdf_single.plot(ax=ax, markersize=40, color='r')
plt.xlim(target_pt[0]-12, target_pt[0]+12)
plt.ylim(target_pt[1]-12, target_pt[1]+12)

plt.show()

In [ ]:
# Buffer plot, this is how the zonalstats method works
gdf_single = gdf.copy().loc[0:2]
gdf_single.loc[0,'coords'] = Point(target_pt[0], target_pt[1])
gdf_single_buff = gdf_single.copy()
gdf_single_buff.geometry = gdf_single.coords.buffer(10)
rstrzed = rio.features.rasterize(gdf_single_buff.geometry.loc[0:1], SJER_chm_data.shape, fill=0, transform=sjer_chm_meta['transform'])
rstrzed = np.ma.masked_where(rstrzed == 0, rstrzed)
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(SJER_chm_data,
          # Here you must set the spatial extent or else the data will not line up with your geopandas layer
          extent=plotting_extent(ds),
          cmap='Greys')
ax.imshow(rstrzed,
         extent=plotting_extent(ds), alpha=0.5, cmap='tab10')

gdf_single_buff.plot(ax=ax, facecolor='none', edgecolor='r', linewidth=2)
gdf_single.plot(ax=ax, markersize=40, color='r')
plt.xlim(target_pt[0]-12, target_pt[0]+12)
plt.ylim(target_pt[1]-12, target_pt[1]+12)

plt.show()

In [ ]:
# Figure for scanline
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(SJER_chm_data,
          # Here you must set the spatial extent or else the data will not line up with your geopandas layer
          extent=plotting_extent(ds),
          cmap='Greys')

scanline_pts = scanline_points(target_pt, 10, [1,1])

for y in scanline_pts[2][1:-1]:
    plt.plot([-100000, 1000000], [y,y], color='b', linestyle='-', linewidth=2)

plt.scatter(scanline_pts[0], scanline_pts[2], color='b')
plt.scatter(scanline_pts[1], scanline_pts[2], color='b')



gdf_single_buff.plot(ax=ax, facecolor='none', edgecolor='r', linewidth=2);
gdf_single.plot(ax=ax, markersize=40, color='r')
plt.xlim(target_pt[0]-13, target_pt[0]+13)
plt.ylim(target_pt[1]-13, target_pt[1]+13)

plt.show()